In [1]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-malay-function-call/resolve/main/function-calls.jsonl
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-malay-function-call/resolve/main/function-calls-complex.jsonl

In [2]:
from malaysian_sft import accept as accept_fn

In [3]:
break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'gay',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI'
]

In [4]:
import json
import random
from glob import glob
from tqdm import tqdm

In [5]:
templates = [
    'tukar ke JSON berdasarkan schema {schema}, teks `{text}`',
    'text `{text}`, convert to JSON using schema {schema}',
    'teks: {text}\n\ntukar ke JSON using schema {schema}',
    'convert to JSON using schema {schema}\n\ntext: {text}',
    '{text}\n\nJSON berdasarkan schema {schema}',
    'JSON berdasarkan schema {schema}\n\n{text}',
]

In [6]:
data = []

with open('function-calls.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        texts = []
        failed = False
        for no, d in enumerate(l['conversations']):

            if d['role'] == 'user':
                t = '<manusia>: '
                c = 'content'
            else:
                if 'functioncall' in d['content']:
                    c = 'content'
                else:
                    if 'functioncall' in l['conversations'][no - 1]['content']:
                        continue
                    c = 'content'
                t = '<bot>: '

            if d[c] is None:
                break

            if 'functioncall' in d[c]:
                d[c] = d[c].split('FUNCTION RESPONSE:')[0].strip()

            if '<functioncall>' in d[c]:
                d[c] = d[c].split('<functioncall>')[1].strip()
                try:
                    d[c] = eval(d[c])
                    d[c] = json.dumps(d[c]['arguments'])
                    
                except:
                    failed = True
                    continue

            n = d[c]
            n = n.strip()

            if len(n) < 3:
                break

            if d['role'] == 'user':
                n = random.choice(templates).format(schema = l['function_call'], text = n)
            t = t + n
            texts.append(t)

        if failed:
            continue

        if not len(texts):
            continue
        while len(texts) and texts[-1].startswith('<manusia>'):
            texts = texts[:-1]
        if not len(texts):
            continue
        if not texts[0].startswith('<manusia>'):
            continue
            
        if not len(texts):
            continue

        data.append({
            'prompt_input': None,
            'input': '\n'.join(texts[:2]),
            'output': None,
        })
            
len(data)

179450it [00:05, 34435.71it/s]


178576

In [7]:
data_complex = []

with open('function-calls-complex.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        texts = []
        failed = False
        for no, d in enumerate(l['conversations']):

            if d['role'] == 'user':
                t = '<manusia>: '
                if random.random() > 0.5:
                    c = 'content'
                else:
                    c = 'content'
            else:
                if 'functioncall' in d['content']:
                    c = 'content'
                else:
                    if 'functioncall' in l['conversations'][no - 1]['content']:
                        continue
                    c = 'content'
                t = '<bot>: '

            if d[c] is None:
                break

            if 'functioncall' in d[c]:
                d[c] = d[c].split('FUNCTION RESPONSE:')[0].strip()

            if '<functioncall>' in d[c]:
                d[c] = d[c].split('<functioncall>')[1].strip()
                try:
                    d[c] = eval(d[c])
                    d[c] = json.dumps(d[c]['arguments'])
                except:
                    failed = True
                    continue


            n = d[c]
            n = n.strip()

            if len(n) < 3:
                break

            if d['role'] == 'user':
                n = random.choice(templates).format(schema = l['function_call'], text = n)
            t = t + n
            texts.append(t)

        if failed:
            continue

        if not len(texts):
            continue
        while len(texts) and texts[-1].startswith('<manusia>'):
            texts = texts[:-1]
        if not len(texts):
            continue
        if not texts[0].startswith('<manusia>'):
            continue
            
        if not len(texts):
            continue

        data_complex.append({
            'prompt_input': None,
            'input': '\n'.join(texts[:2]),
            'output': None,
        })
            
len(data_complex)

24977

In [8]:
# !pip3.8 install to-json-schema

In [9]:
from to_json_schema.to_json_schema import SchemaBuilder

schema_builder = SchemaBuilder()

different_schema = []
files = [
    'function-calls-complex.jsonl',
    'function-calls.jsonl'
]
for filename in files:
    with open(filename) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            texts = []
            failed = False
            for no, d in enumerate(l['conversations']):

                if d['role'] == 'user':
                    t = '<manusia>: '
                    c = 'content'
                else:
                    if 'functioncall' in d['content']:
                        c = 'content'
                    else:
                        if 'functioncall' in l['conversations'][no - 1]['content']:
                            continue
                        c = 'content'
                    t = '<bot>: '

                if d[c] is None:
                    break

                if 'functioncall' in d[c]:
                    d[c] = d[c].split('FUNCTION RESPONSE:')[0].strip()

                if '<functioncall>' in d[c]:
                    d[c] = d[c].split('<functioncall>')[1].strip()
                    try:
                        d[c] = eval(d[c])
                        d[c] = json.dumps(d[c]['arguments'])

                    except:
                        failed = True
                        continue

                n = d[c]
                n = n.strip()

                if len(n) < 3:
                    break

                t = t + n
                texts.append(t)

            if failed:
                continue

            if not len(texts):
                continue
            while len(texts) and texts[-1].startswith('<manusia>'):
                texts = texts[:-1]
            if not len(texts):
                continue
            if not texts[0].startswith('<manusia>'):
                continue

            texts = texts[:2]
            if not len(texts):
                continue

            f = json.loads(texts[1].split('>: ')[1])
            if 'complex' not in filename and random.random() > 0.6:
                keys = f.keys()
                keys = [f"'{k}'" for k in keys]
                schema = '{' + ', '.join(keys) + '}'
            else:
                schema = schema_builder.to_json_schema(f)

            n = texts[0].split('>: ')[1]
            n = random.choice(templates).format(schema = schema, text = n)
            texts[0] = f'<manusia>: {n}'

            different_schema.append({
                'prompt_input': None,
                'input': '\n'.join(texts),
                'output': None,
            })
        
len(different_schema)

24986it [00:01, 18298.73it/s]
179450it [00:07, 23888.82it/s]


203553

In [14]:
def generate_and_tokenize_prompt(row):
    texts = ['<s>']

    if 'function_call' in row:
        t = row['function_call']
        texts.append(f'\n[FUNCTIONCALL]\n{t}\n')

    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                try:
                    human = splitted[i].split('<manusia>:')[1]
                except:
                    continue
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human.strip())
            outputs.append(bot.strip())
    else:
        inputs = [row['input']]
        outputs = [row['output']]

    return inputs, outputs

In [17]:
from tqdm import tqdm

all_data = []
for d in tqdm(data + data_complex + different_schema):
    r = generate_and_tokenize_prompt(d)
    
    if not accept_fn(r[0][0]):
        continue
    
    if not accept_fn(r[1][0]):
        continue
    
    all_data.append({
        'prompt_input': None,
        'input': r[0][0],
        'output': r[1][0],
    })

100%|█████████████████████████████████| 407106/407106 [02:02<00:00, 3318.39it/s]


In [18]:
len(all_data)

58177

In [19]:
import pandas as pd

df = pd.DataFrame(all_data)
df.head()

,prompt_input,input,output
0,None,"tukar ke JSON berdasarkan schema {\n ""name"": ""...","{""numbers"": [5, 2, 9, 1, 7, 4, 6, 3, 8]}"
1,None,"Nombor-nombor adalah 8, 3, 5, 2, 7, 4, 9, 1, 6...","{""numbers"": [8, 3, 5, 2, 7, 4, 9, 1, 6]}"
2,None,"convert to JSON using schema {\n ""name"": ""rese...","{""location"": ""Kuala Lumpur"", ""check_in_date"": ..."
3,None,"teks: I'd like to order two nasi lemak, extra ...","{""item"": ""nasi lemak"", ""quantity"": 2, ""special..."
4,None,"Hai, boleh cari resepi spaghetti carbonara unt...","{""masakan"": ""spaghetti carbonara"", ""bahan"": ""t..."


In [20]:
df.shape

(58177, 3)

In [22]:
df.to_parquet('prepared-force-json-format.parquet')

In [23]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-force-json-format.parquet",
    path_in_repo='data/force_json_format-00000-of-00001.parquet',
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

prepared-force-json-format.parquet:   0%|          | 0.00/12.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-SFT/commit/37b7528801cba578e9fffcd45db0550ea5b3fd50', commit_message='Upload data/force_json_format-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='37b7528801cba578e9fffcd45db0550ea5b3fd50', pr_url=None, pr_revision=None, pr_num=None)